In [ ]:
from pathlib import Path
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
df = {}
for split in ['ftqa_train', 'ftqa_val']:
    mapping_path='/data/{}.xlsx'.format(split)
    mapping = pd.read_excel(mapping_path)
    df[split] = mapping.copy()

In [3]:
def find_wh(x):
    doc = nlp(x)
    for token in doc:
        if token.tag_ in ['WDT', 'WP', 'WP$', 'WRB']:
            qword = token.text[0].upper() + token.text[1:]
            if qword == 'Whose' or qword == 'Whom':
                qword = 'Who'
            elif qword not in ['What', 'How', 'Who', 'Why', 'Where', 'When', 'Which']:
                qword = 'None'
            return qword
    return 'None'

In [ ]:
for i in df.keys():
    df[i]['wh'] = df[i]['question'].apply(lambda x: find_wh(x))
    df[i] = df[i][df[i]['wh'] != 'None']

In [ ]:
for i in df.keys():
    set = i.split('_')[-1]
    name = '_'.join(i.split('_')[0:-1])
    df[i].to_csv('/data/{}_wh_{}.xlsx'.format(name,set), index=False)